# Todo

> Todo app

In [ ]:
#| default_exp todo

In [ ]:
#| export
from fasthtml.common import *
from functools import partial
from fasthtml.jupyter import *
from monsterui.all import *
import ast, os
from ghapi.all import GhApi
from datetime import datetime, timedelta
from web.utils import *
api = GhApi(token=os.environ['GITHUB_TOKEN'])

In [ ]:
#| export
ar = APIRouter(prefix='/todo', body_wrap=layout)

In [ ]:
#| export
def populate_todos(n_days=30):
    me = api.users.get_authenticated()
    username = me.login
    
    time_range = (datetime.now() - timedelta(days=n_days)).strftime('%Y-%m-%d')
    watched_repos = [o.full_name for o in api.activity.list_repos_watched_by_user(username)]
    watched_issues = api.search.issues_and_pull_requests(q=f'repo:{" repo:".join(watched_repos)} state:open updated:>={time_range}')
    involved_issues = api.search.issues_and_pull_requests(q=f'involves:{username} state:open updated:>={time_range}')
    
    for search_result in (watched_issues, involved_issues):
        for item in search_result['items']:
            issue = Todo(
                        id=item['id'], 
                        title=item['title'],
                        updated_at=item['updated_at'],
                        labels=','.join(label['name'] for label in item['labels']),
                        repo_url=item['repository_url'],
                        html_url=item['html_url'],
                        repo_owner=item['repository_url'].split('/')[-2],
                        repo_name=item['repository_url'].split('/')[-1],
                        number=item['number'])
            if item['id'] in db.todos: db.todos.update(issue)
            else: db.todos.insert(issue)

In [ ]:
#| export
def TodoCard(issue:Todo):  
    return Card(
        A(issue.title, href=issue.html_url, cls=AT.primary, target='_blank'),
        DivFullySpaced(
            A(Subtitle(f"{issue.repo_owner}/{issue.repo_name}"), href=issue.repo_url, cls=AT.muted, target='_blank'),
            Time(issue.updated_at, cls=TextT.sm)),
        DivLAligned(*map(Label, [o for o in issue.labels.split(',') if o != ''])),
    footer=Grid(*[Button(s, hx_post=update_status.to(id=issue.id, new_status=s), hx_target='body') for s in status_opts], cols_max=3))

In [ ]:
#| export
@ar    
def update_status(id:str, new_status:str): 
    db.todos.upsert(Todo(id=id, status=new_status))
    return index()

In [ ]:
if IN_NOTEBOOK: 
    db = database('../productivity_app.db')
    db.todos = db.create(Todo) 

In [ ]:
#| export
@ar
def index(request=None):
    if request and 'hx-request' not in request.headers:
        last_update = db.execute('select max(updated_at) from todo').fetchone()[0]
        if last_update:
            last_update = datetime.strptime(last_update, '%Y-%m-%dT%H:%M:%SZ').date()
            populate_todos((datetime.today().date() - last_update).days + 1)
        else:
            populate_todos(30)
    
    cutoff = (datetime.now() - timedelta(days=180)).strftime('%Y-%m-%dT%H:%M:%SZ')
    base_query = f'updated_at>=?'

    def _todos(s): return db.todos('updated_at>=? and status=?', [cutoff, s])
    unassigned = db.todos(f'updated_at>=? AND status IS NULL', [cutoff])
    
    return Container(
        Section(H2("Unassigned"), Grid(*map(TodoCard, unassigned), cols_max=3)) if unassigned else None,
        Section(
            Grid(
                Div(H2("Todo"),        *map(TodoCard, _todos('Todo')),        cls='space-y-3'), 
                Div(H2("In Progress"), *map(TodoCard, _todos('In Progress')), cls='space-y-3'), 
                Div(H2("Done"),        *map(TodoCard, _todos('Done')),        cls='space-y-3'))),
        Section(Details(Summary("Someday"), Grid(*map(TodoCard, _todos('Someday')), cols_max=3))),   
        Section(Details(Summary("Other"),   Grid(*map(TodoCard, _todos('Other')),   cols_max=3))),   
        Section(Details(Summary("Archived"),Grid(*map(TodoCard, _todos('Archived')),cols_max=3))))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()